## Libraries

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from apyori import apriori
import datetime
import statsmodels.api as sm
from scipy.stats import spearmanr 
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import cdist 
import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from boruta import BorutaPy

from sklearn import linear_model
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


import joblib

from math import radians, cos, sin, asin, sqrt
from numpy import mean
from numpy import std

# write all imported libraries to requirements.txt
# !pipreqs --force .

In [ ]:
dataframe = pd.read_csv('./dataset.csv')
dataframe

In [ ]:
dataframe.isnull().sum()

# A look at the data

In [ ]:
# plot nan values count for each column
dataframe.isnull().sum().plot(kind='bar', figsize=(10, 5))
plt.title('Count of NaN values for each column')
plt.xlabel('Columns')
plt.ylabel('Count of NaN values')
plt.show()

### Association Rule Mining

In [ ]:
df = dataframe[[
  "Gender",
  "With_Kids",
  "Basket_Size",
  "Attire",
  "Wash_Item",
  "shirt_type",
]]

df

In [ ]:
col_list = df.columns
df_oh = df[col_list]
df = df.drop(col_list, 1)
df_oh = pd.get_dummies(df_oh)
df = pd.concat([df, df_oh], axis=1)
df.shape

In [ ]:
records = []
for i in range(0, 4000):
    records.append([str(df.columns[j]) for j in range(0, 13) if df.values[i,j] == 1])
    
records[0]

In [ ]:
association_results = apriori(records)
association_results = list(association_results)
len(association_results)

In [ ]:
cnt = 0

association_r_df = pd.DataFrame(columns=['Items', 'Support', 'Confidence', 'Lift'])

for item in association_results:
    cnt += 1
    # first index of the inner list
    # Contains base item and add item
    pair = item[0]
    items = [x for x in pair]
    if len(items) < 2:
        association_r_df = association_r_df.append({'Items':items[0], 'Support':item[1], 'Confidence':item[2][0][2], 'Lift':item[2][0][3]}, ignore_index=True)
        print("(Rule " + str(cnt) + ") " + items[0])
    else:
        association_r_df = association_r_df.append({'Items':items[0] + " -> " + items[1], 'Support':item[1], 'Confidence':item[2][0][2], 'Lift':item[2][0][3]}, ignore_index=True)
        print("(Rule " + str(cnt) + ") " + items[0] + " -> " + items[1])
    
    # second index of the inner list
    print("Support: " + str(round(item[1],3)))
    
    # third index of the list located at 0th
    # of the third index of the inner list 
    
    print("Confidence: " + str(round(item[2][0][2],4)))
    print("Lift: " + str(round(item[2][0][3],4)))
    print("=====================================")
    
display(association_r_df)
association_r_df.to_csv('association_rules.csv')

# Data Preprocessing

In [ ]:
p_df = dataframe.copy()

print(p_df.isnull().sum())
len(p_df)

In [ ]:
# p_df = p_df.dropna(subset=['Race', 'Gender', 'latitude', 'longitude'], inplace=False)

# p_df['Body_Size'].fillna('moderate', inplace=True)
# p_df['Age_Range'].fillna(p_df['Age_Range'].mean(), inplace=True)
# p_df["With_Kids"].fillna("no", inplace=True)
# p_df['Kids_Category'].fillna('no_kids', inplace=True)
# p_df['Basket_Size'].fillna('small', inplace=True)
# p_df['TimeSpent_minutes'].fillna(p_df['TimeSpent_minutes'].mean(), inplace=True)
# p_df['buyDrinks'].fillna(0.0, inplace=True)
# p_df['TotalSpent_RM'].fillna(0.0, inplace=True)

# imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# imputed_data = imp.fit_transform(p_df)

# imp_df = pd.DataFrame(imputed_data, columns = p_df.columns)

# display(imp_df)
# imp_df.isnull().sum()

# # plot nan values count for each column
# imp_df.isnull().sum().plot(kind='bar', figsize=(10, 5))
# plt.title('Count of NaN values for each column')
# plt.xlabel('Columns')
# plt.ylabel('Count of NaN values')
# plt.show()

In [ ]:
# weather_data = pd.read_csv('./weather.csv')
# weather_data = weather_data.drop(['Unnamed: 0'], axis=1)
# weather_data.tail(12)

In [ ]:
# # add weather data to the dataframe
# weather_data['date'] = weather_data['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

# # add weather column to imp_df and fill with NaN
# imp_df['weather'] = np.nan

# display(imp_df)

# hour_index = ['0', '3', '6', '9', '12', '15', '18', '21']

# # loop imp_df
# for i in range(len(imp_df)):
#     # loop weather_data
#     for j in range(len(weather_data)):
#         # check if date is the same
#         if imp_df['Date'][i] == weather_data['date'][j] :
#           if int(datetime.datetime.strptime(imp_df['Time'][i], '%H:%M:%S').strftime('%H')) == str(weather_data['hour'][j]).replace('00', ''):
#             imp_df['weather'][i] = weather_data['desc'][j]
#           else:
#             actual_hour = int(datetime.datetime.strptime(imp_df['Time'][i], '%H:%M:%S').strftime('%H'))
#             for k in range(len(hour_index)):
#               is_in_between = False
#               if k == len(hour_index) - 1:
#                 is_in_between = actual_hour > int(hour_index[k])
#               else: 
#                 is_in_between = actual_hour > int(hour_index[k]) and actual_hour < int(hour_index[k+1])
#               if is_in_between and k != len(hour_index) - 1:
#                 lower_bound = hour_index[k]
#                 upper_bound = hour_index[k+1]
#                 lower_bound_offset = abs(actual_hour - int(lower_bound))
#                 upper_bound_offset = abs(actual_hour - int(upper_bound))
#                 if lower_bound_offset < upper_bound_offset:
#                   imp_df['weather'][i] = weather_data['desc'][j]
#                 else:
#                   imp_df['weather'][i] = weather_data['desc'][j+1]
#               else:
#                 imp_df['weather'][i] = weather_data['desc'][j]

# display(weather_data)
# display(imp_df)

# imp_df['Day'] = imp_df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').strftime('%A'))
# imp_df['Month'] = imp_df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').strftime('%B'))
# imp_df['Year'] = imp_df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').strftime('%Y'))
# imp_df['Time'] = imp_df['Time'].apply(lambda x: x.replace(';', ':'))
# display(imp_df)
# imp_df['Time'] = imp_df['Time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').strftime('%H'))
# imp_df['Time'] = imp_df['Time'].apply(lambda x: 'Morning' if int(x) in range(6,12) else 'Afternoon' if int(x) in range(12,18) else 'Evening' if int(x) in range(18,24) else 'Night')

In [ ]:
# save to csv
# imp_df.to_csv('dataset_w_weather.csv', index=False)

### Add RWI (Relative Wealth Index) to dataset

In [ ]:
# imp_df = pd.read_csv('dataset_w_weather.csv')
# rwi_dataset = pd.read_csv('./scraper/mys_relative_wealth_index.csv')

# rwi_df = pd.DataFrame(rwi_dataset)
# rwi_df = rwi_df.dropna(subset=['latitude', 'longitude', 'rwi'], inplace=False)
# rwi_df

In [ ]:
# def haversine(lon1, lat1, lon2, lat2):
#     """
#     Calculate the great circle distance in kilometers between two points 
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians 
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

#     # haversine formula 
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * asin(sqrt(a)) 
#     r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
#     return c * r

In [ ]:
# imp_df['rwi'] = np.nan
 
# for i in range(len(imp_df)):
#   # calculate every distance 
#   distance = []
#   for j in range(len(rwi_df)):
#     distance.append(haversine(imp_df['longitude'][i], imp_df['latitude'][i], rwi_df['longitude'][j], rwi_df['latitude'][j]))
  
#   # get the index of the minimum distance
#   min_index = distance.index(min(distance))
#   imp_df['rwi'][i] = rwi_df['rwi'][min_index]

In [ ]:
# # save to csv
# imp_df.to_csv('dataset_w_weather&rwi.csv', index=False)

# imp_df

#### THE ABOVE CODES ARE USED TO CLEAN AND MERGE EXTERNAL DATASET TO THE ORIGINAL DATASET, THE RESULTING DATASET IS SAVED TO dataset_w_weather&rwi&city.csv THUS THE CODES ARE ALL COMMENTED OUT

# Explanatory Data Analysis

In [ ]:
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
imp_df.dtypes

In [ ]:
for col in imp_df.columns:
  print(col, imp_df[col].nunique())

In [ ]:
ll = imp_df.copy()
print(len(ll))
# find unique combinations of latitude and longitude
ll['lat_long'] = ll['latitude'].astype(str) + '_' + ll['longitude'].astype(str)
ll['lat_long'].unique()
len(ll['lat_long'].unique())

In [ ]:
# loop all columns
print("For Washer NO: ")
print('===================================== \n')
for col in imp_df.columns:
    col_1 = imp_df.loc[1:100,[col]]
    target = imp_df.loc[1:100,['Washer_No']]
    spearmanr_coef, p_value = spearmanr(col_1, target)
    print("Column: ", col)
    print ("coefficient=", spearmanr_coef)
    print('p-value=', p_value)
    print('===================================== \n')
    
print("For Dryer NO: ")
print('===================================== \n')
for col in imp_df.columns:
    col_1 = imp_df.loc[1:100,[col]]
    target = imp_df.loc[1:100,['Dryer_No']]
    spearmanr_coef, p_value = spearmanr(col_1, target)
    print("Column: ", col)
    print ("coefficient=", spearmanr_coef)
    print('p-value=', p_value)
    print('===================================== \n')
    
print("For Total Spent: ")
print('===================================== \n')
for col in imp_df.columns:
    col_1 = imp_df.loc[1:100,[col]]
    target = imp_df.loc[1:100,['TotalSpent_RM']]
    spearmanr_coef, p_value = spearmanr(col_1, target)
    print("Column: ", col)
    print ("coefficient=", spearmanr_coef)
    print('p-value=', p_value)
    print('===================================== \n')

In [ ]:
imp_df = imp_df.apply(LabelEncoder().fit_transform)
sns.set(rc={'figure.figsize':(12, 12)})
correlation_matrix = imp_df.iloc[:,:].corr().round(1)
sns.heatmap(data=correlation_matrix, annot=True)
plt.savefig('correlation_matrix.png')

In [ ]:
imp_df['weather'].value_counts()

In [ ]:

plt.style.use('fivethirtyeight')
plt.figure(figsize=(18,4))
weather_dist_data = pd.read_csv('./dataset_w_weather&rwi&city.csv')
sns.countplot(x='weather', data=weather_dist_data)
plt.show()
plt.savefig('weather_distribution.png')

In [ ]:
# plot relationship between weather and washer no
plt.figure(figsize=(16,4))
sns.countplot(x='weather', hue='Washer_No', data=imp_df)
plt.show()

In [ ]:
# plot relationship between weather and dryer no
plt.figure(figsize=(16,4))
sns.countplot(x='weather', hue='Dryer_No', data=imp_df)
plt.show()

In [ ]:
# plot relationship between weather and total spent
plt.figure(figsize=(16,4))
sns.countplot(x='weather', hue='TotalSpent_RM', data=imp_df)
plt.show()
plt.savefig('weather_totalspent.png')

In [ ]:
# plot distribution of rwi 
plt.figure(figsize=(12,4))
sns.distplot(imp_df['rwi'])
plt.show()
plt.savefig('rwi_distribution.png')

In [ ]:
# plot boxplot for rwi
plt.figure(figsize=(5,5))
sns.boxplot(imp_df['rwi'])
plt.show()
plt.savefig('rwi_boxplot.png')

In [ ]:
# make a box plot for every column
boxplottingdf = imp_df.copy()
boxplottingdf = boxplottingdf.apply(LabelEncoder().fit_transform)
for col in boxplottingdf.columns:
  plt.figure(figsize=(5,5))
  sns.boxplot(boxplottingdf[col])
  plt.show()


In [ ]:
# plot relationship between rwi and total spent
plt.figure(figsize=(20,4))

sns.regplot(x='rwi', y='TotalSpent_RM', data=imp_df)
plt.show()

coordinate = plt.figure(figsize=(8,8))
coordinate = plt.scatter(imp_df['longitude'],imp_df['latitude'])
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.show()

In [ ]:
# plot relationship between attire and total spent
plt.figure(figsize=(16,4))

fig, ax = plt.subplots(1,2, figsize=(16,4))
sns.countplot(x='Attire', hue='Washer_No', data=imp_df, ax=ax[0])
sns.countplot(x='Attire', hue='Dryer_No', data=imp_df, ax=ax[1])
plt.show()

In [ ]:
plt.figure(figsize=(18,10))

fig, ax = plt.subplots(2,2, figsize=(18,10))
sns.countplot(x='Time', hue='Washer_No', data=imp_df, ax=ax[0, 0], palette='Set1')
sns.countplot(x='Time', hue='Dryer_No', data=imp_df, ax=ax[0, 1])
sns.countplot(x='Time', hue='TotalSpent_RM', data=imp_df, ax=ax[1, 0])

plt.show()

In [ ]:
# relationship between time spent in minutes and total spent and buy drinks
plt.figure(figsize=(10,10))
fig, (ax1, ax2) = plt.subplots(2,2, figsize=(18,10))
sns.regplot(x='TimeSpent_minutes', y='TotalSpent_RM', data=imp_df, ax=ax1[0])
sns.regplot(x='TimeSpent_minutes', y='buyDrinks', data=imp_df, ax=ax1[1])
tsm_w = imp_df.apply(LabelEncoder().fit_transform)
sns.regplot(x='rwi', y='TotalSpent_RM', data=imp_df, ax=ax2[0])
sns.regplot(x='rwi', y='buyDrinks', data=imp_df, ax=ax2[1])

plt.show()

In [ ]:
plt.figure(figsize=(18,6))
sns.countplot(hue='TimeSpent_minutes', x='Wash_Item', data=imp_df)
plt.show()
plt.savefig('washitem_timespent.png')

average_time = imp_df.groupby('Wash_Item')['TimeSpent_minutes'].mean()
#plot avarge time spent for each wash item
plt.figure(figsize=(10,4))
average_time.plot(kind='bar')
plt.show()
plt.savefig('average_time-by_washItem.png')


In [ ]:
plt.figure(1 , figsize = (8 , 4))
sns.countplot(y = 'Gender' , data = imp_df)
plt.show()

In [ ]:
plt.figure(1 , figsize = (8 , 4))
sns.countplot(y = 'Time' , data = imp_df)
plt.show()

In [ ]:
plt.figure(1 , figsize = (8 , 4))
sns.countplot(y = 'Day' , data = imp_df)
plt.show()

In [ ]:
plt.figure(1 , figsize = (18 , 6))
n = 0 
for x in [ 'Age_Range', 'Washer_No', 'Dryer_No']:
    n += 1
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace =0.5 , wspace = 0.5)
    sns.distplot(dataframe[x] , bins = 20)
    plt.title('Distplot of {}'.format(x))
    plt.savefig( 'Distplot of {}'.format(x))
plt.show()

In [ ]:
coordinate = plt.figure(figsize=(8,8))
coordinate = plt.scatter(imp_df['longitude'],imp_df['latitude'])
plt.xlabel('longitude')
plt.ylabel('latitude')
coordinate.figure.savefig('Coordinate.png')
coordinate

In [ ]:
age_washer = plt.figure(figsize=(14,6))
age_washer = plt.scatter(imp_df['rwi'],imp_df['TotalSpent_RM'])
plt.xlabel('rwi')
plt.ylabel('total spent')
age_washer.figure.savefig('rwi_spent.png')
age_washer

In [ ]:
age_washer = plt.figure(figsize=(14,6))
age_washer = plt.scatter(imp_df['rwi'],imp_df['Age_Range'])
plt.xlabel('rwi')
plt.ylabel('Age Range')
age_washer.figure.savefig('rwi_age.png')
age_washer

In [ ]:
# reorder the time columns
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
display(imp_df.head())

imp_df['Time'] = pd.Categorical(imp_df['Time'], categories=['Morning', 'Afternoon', 'Evening', 'Night'], ordered=True)
plt.figure(figsize=(10, 10))
pd.crosstab(imp_df['Time'], imp_df['Race'] ).plot()
plt.savefig('relationship-between-race-and-time.png')

In [ ]:
fig = plt.figure(figsize =(10, 7))

# Creating plot
plt.boxplot(imp_df['Age_Range'])

plt.ylabel('Age_Range')
# show plot
plt.show()
plt.savefig('boxplot-age.png')

#### What types of customers will likely to choose Washer No.2 and Dryer No.3

In [ ]:
sim = imp_df.copy()
display(sim)

display(sim['Washer_No'].value_counts())
display(sim['Dryer_No'].value_counts())

f_1 = sim['Washer_No'] == 3
f_2 = sim['Dryer_No'] == 10
customers = sim[f_1 & f_2].iloc[:,2:16]

for column in customers.columns:
    print(column + ': ' + str(customers[column].max()))
    
with open('washer_dryer_pred.txt', 'w') as f:
    f.write('Washer_No: 2 and Dryer_No: 3' + '\n\n')
    for column in customers.columns:
      f.write(column + ': ' + str(customers[column].max()) + '\n')


In [ ]:
imp_df['TotalSpent_RM'].plot(kind="hist", figsize=(10,8))

# Modelling

## Linear Regression Models

## Regression for Age Range

In [ ]:
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
imp_df.head(5)

#### Feature Selection (LR - Age Range) *using boruta and rfe

In [ ]:
ar_fs_df = imp_df.copy()
ar_fs_df = ar_fs_df.apply(LabelEncoder().fit_transform)

ar_x = ar_fs_df.drop(['Age_Range'], axis=1)
ar_y = ar_fs_df['Age_Range']

display(ar_x.head())

In [ ]:
# Boruta
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))
  
ar_fs_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
ar_fs = BorutaPy(ar_fs_rf, n_estimators="auto", random_state=1)
ar_fs.fit(ar_x.values, ar_y.values.ravel())
ar_boruta_score = ranking(list(map(float, ar_fs.ranking_)), ar_x.columns, order=-1)
ar_boruta_score = pd.DataFrame(list(ar_boruta_score.items()), columns=['Features', 'Score'])
ar_boruta_score = ar_boruta_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(ar_boruta_score.head(10))

print('---------Bottom 10----------')
display(ar_boruta_score.tail(10))

sns_boruta_plot = sns.catplot(x="Score", y="Features", data = ar_boruta_score[:10], kind = "bar", height=14, aspect=2, palette='coolwarm')
plt.title("Age Range LR Boruta Top 10 Features")
sns_boruta_plot.figure.savefig('Age_Range_LR_Boruta.png')

pd.DataFrame.to_csv(ar_boruta_score.head(10), 'Age_Range_LR_Boruta.csv')

# RFE
ar_fs_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5, n_estimators=100)
ar_fs_rf.fit(ar_x, ar_y)
ar_rfe = RFECV(ar_fs_rf, min_features_to_select = 1, cv = 3)
ar_rfe.fit(ar_x, ar_y)

ar_rfe_score = ranking(list(map(float, ar_rfe.ranking_)), ar_x.columns, order=-1)
ar_rfe_score = pd.DataFrame(list(ar_rfe_score.items()), columns=['Features', 'Score'])
ar_rfe_score = ar_rfe_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(ar_rfe_score.head(10))

print('---------Bottom 10----------')
display(ar_rfe_score.tail(10))
pd.DataFrame.to_csv(ar_rfe_score.head(10), 'AgeRange_RFE_Top10.csv')

sns_wi_rfe_plot = sns.catplot(x="Score", y="Features", data = ar_rfe_score[0:10], kind = "bar", height=14, aspect=1.9, palette='coolwarm')
plt.title("Age Range RFE Top-10 Features")
sns_wi_rfe_plot.figure.savefig('AgeRange_RFE.png')

In [ ]:
ar_lrX = ar_x[["rwi", "TotalSpent_RM","Time", "city", "TimeSpent_minutes"]]
ar_lrY = ar_y

ar_lr_X_train, ar_lr_X_test, ar_lr_y_train, ar_lr_y_test = train_test_split(ar_lrX, ar_lrY, test_size = 0.2, random_state = 0)

print(ar_lr_X_train.shape)
print(ar_lr_X_test.shape)


### Linear Regression

In [ ]:

ar_reg = linear_model.LinearRegression()
ar_lr_model = ar_reg.fit(ar_lr_X_train,ar_lr_y_train)  

print ("coefficients : ",ar_reg.coef_) #Slope
print ("Intercept : ",ar_reg.intercept_)

ar_lr_model.score(ar_lr_X_train,ar_lr_y_train)

def get_regression_predictions(input_features,intercept,slope):
    predicted_values = input_features*slope + intercept
    return predicted_values

y_pred = ar_reg.predict(ar_lr_X_test)
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_pred - ar_lr_y_test)))
print("Mean sum of squares (MSE): %.2f" % np.mean((y_pred - ar_lr_y_test)** 2))
print("R2-score: %.2f" % r2_score(y_pred , ar_lr_y_test) )

# plot slope 
plt.figure(figsize=(10,8))
plt.plot(ar_lr_X_train, ar_lr_y_train, 'o', label='original data', color='black')
plt.plot(ar_lr_X_train, ar_reg.coef_*ar_lr_X_train + ar_reg.intercept_, 'r', label='fitted line', color='red')
plt.legend()
plt.show()

filename = 'AgeRange_LR.sav'
pickle.dump(ar_reg, open(filename, 'wb'))

### Support Vector Regression

In [ ]:
ar_svr = SVR(kernel = 'rbf')

ar_svr.fit(ar_lr_X_train, ar_lr_y_train)
y_pred = ar_svr.predict(ar_lr_X_test)

print("Mean Absolute Error: {:.3f}".format(mean_absolute_error(ar_lr_y_test, y_pred)))
print("Mean Squared Error: {:.3f}".format(mean_squared_error(ar_lr_y_test, y_pred)))
print("R2 Score: {:.3f}".format(r2_score(ar_lr_y_test, y_pred)))
print("Accuracy: {:.3f}".format(ar_svr.score(ar_lr_X_test, ar_lr_y_test)))

gammas = [0.1, 1, 10, 100]
svr_mae = [] 
svr_acc = []
scr_mse = []
for gamma in gammas:
    ar_svr = SVR(kernel = 'rbf', gamma = gamma)
    ar_svr.fit(ar_lr_X_train, ar_lr_y_train)
    svr_y_pred = ar_svr.predict(ar_lr_X_test)
    svr_mae.append(mean_absolute_error(ar_lr_y_test, svr_y_pred))
    svr_acc.append(ar_svr.score(ar_lr_X_test, ar_lr_y_test)) 
    scr_mse.append(mean_squared_error(ar_lr_y_test, svr_y_pred))
    
plt.plot(gammas, svr_mae)
plt.xlabel('gamma')
plt.ylabel('MAE')
plt.title('gamma vs MAE').figure.savefig('AgeRange_SVR_Gamma_vs_Mae.png') 
plt.show()

plt.plot(gammas, svr_acc)
plt.xlabel('gamma')
plt.ylabel('Accuracy')
plt.title('gamma vs Accuracy').figure.savefig('AgeRange_SVR_Gamma_vs_Accuracy.png')
plt.show()

plt.plot(gammas, scr_mse)
plt.xlabel('gamma')
plt.ylabel('MSE')
plt.title('gamma vs MSE').figure.savefig('AgeRange_SVR_Gamma_vs_MSE.png')
plt.show()

# save the model to disk
filename = 'AgeRange_SVR.sav'
pickle.dump(ar_svr, open(filename, 'wb'))

### Desicion Tree Regressor

In [ ]:
ar_dtrX = ar_x
ar_dtrY = ar_y
display(ar_dtrX.head())
ar_dtr_X_train, ar_dtr_X_test, ar_dtr_y_train, ar_dtr_y_test = train_test_split(ar_dtrX, ar_dtrY, test_size = 0.2, random_state = 0)

print(ar_dtr_X_train.shape)
print(ar_dtr_X_test.shape)

ar_dtr = DecisionTreeRegressor(random_state=0, max_depth=7)
ar_dtr.fit(ar_dtr_X_train, ar_dtr_y_train)

# score model
score = ar_dtr.score(ar_dtr_X_test, ar_dtr_y_test)
print(score)

# make predictions
expected = ar_dtr_y_test
predicted = ar_dtr.predict(ar_dtr_X_test)

print("R2-score: %.2f" % r2_score(predicted , expected) )
print("Mean absolute error: %.2f" % np.mean(np.absolute(predicted - expected)))
print("Mean sum of squares (MSE): %.2f" % np.mean((predicted - expected) ** 2))

# save the model to disk
filename = 'AgeRange_DTR.sav'
pickle.dump(ar_dtr, open(filename, 'wb'))

#### Decision Tree Regressor (Tuning)

In [ ]:
depth = 30
max_depth = [i for i in range(1,depth+1)] 
dtr_mae = [] 
for i in range(1,depth+1):
    dtr = DecisionTreeRegressor(max_depth=i)
    dtr.fit(ar_dtr_X_train, ar_dtr_y_train)
    dtr_y_pred = dtr.predict(ar_dtr_X_test)
    dtr_mae.append(mean_absolute_error(ar_dtr_y_test, dtr_y_pred))
    
plt.plot(max_depth, dtr_mae)
plt.xlabel('max_depth')
plt.ylabel('MAE')
plt.title('max_depth vs MAE').figure.savefig('ar_dtr_depth_vs_mae.png') 
plt.show()

## Regression for Number Of Baskets

#### Feature Selection

In [ ]:
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
imp_df.head(5)

nob_fs_df = imp_df.copy()
nob_fs_df = ar_fs_df.apply(LabelEncoder().fit_transform)

nob_x = nob_fs_df.drop(['Num_of_Baskets'], axis=1)
nob_y = nob_fs_df['Num_of_Baskets']

display(ar_x.head())

# Boruta
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

nob_fs_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
nob_fs = BorutaPy(nob_fs_rf, n_estimators="auto", random_state=1)
nob_fs.fit(nob_x.values, nob_y.values.ravel())
nob_boruta_score = ranking(list(map(float, nob_fs.ranking_)), nob_x.columns, order=-1)
nob_boruta_score = pd.DataFrame(list(nob_boruta_score.items()), columns=['Features', 'Score'])
nob_boruta_score = nob_boruta_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(nob_boruta_score.head(10))

print('---------Bottom 10----------')
display(nob_boruta_score.tail(10))
pd.DataFrame.to_csv(nob_boruta_score.head(10), 'NumOfBaskets_Boruta_Top10.csv')

sns_boruta_plot = sns.catplot(x="Score", y="Features", data = nob_boruta_score[:10], kind = "bar", height=14, aspect=2, palette='coolwarm')
plt.title("Num of Baskets LR Boruta Top 10 Features")
sns_boruta_plot.figure.savefig('NOB_LR_Boruta.png')

pd.DataFrame.to_csv(nob_boruta_score.head(10), 'NOB_LR_Boruta.csv')

# RFE
nob_fs_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5, n_estimators=100)
nob_fs_rf.fit(nob_x, nob_y)
nob_rfe = RFECV(nob_fs_rf, min_features_to_select = 1, cv = 3)
nob_rfe.fit(nob_x, nob_y)

nob_rfe_score = ranking(list(map(float, nob_rfe.ranking_)), nob_x.columns, order=-1)
nob_rfe_score = pd.DataFrame(list(nob_rfe_score.items()), columns=['Features', 'Score'])
nob_rfe_score = nob_rfe_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(nob_rfe_score.head(10))

print('---------Bottom 10----------')
display(nob_rfe_score.tail(10))
pd.DataFrame.to_csv(nob_rfe_score.head(10), 'NumOfBaskets_RFE_Top10.csv')

sns_wi_rfe_plot = sns.catplot(x="Score", y="Features", data = nob_rfe_score[0:10], kind = "bar", height=14, aspect=1.9, palette='coolwarm')
plt.title("Num of Baskets RFE Top-10 Features")
sns_wi_rfe_plot.figure.savefig('NOB_RFE.png')

In [ ]:
nob_lrX = nob_x
nob_lrY = nob_y

nob_lr_X_train, nob_lr_X_test, nob_lr_y_train, nob_lr_y_test = train_test_split(nob_lrX, nob_lrY, test_size = 0.2, random_state = 0)

print(nob_lr_X_train.shape)
print(nob_lr_X_test.shape)


### Linear Regression

In [ ]:

nob_reg = linear_model.LinearRegression()
nob_lr_model = nob_reg.fit(nob_lr_X_train,nob_lr_y_train)  

print ("coefficients : ",nob_reg.coef_) #Slope
print ("Intercept : ",nob_reg.intercept_)

nob_lr_model.score(nob_lr_X_train,nob_lr_y_train)

def get_regression_predictions(input_features,intercept,slope):
    predicted_values = input_features*slope + intercept
    return predicted_values

y_pred = nob_reg.predict(nob_lr_X_test)
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_pred - nob_lr_y_test)))
print("Mean sum of squares (MSE): %.2f" % np.mean((y_pred - nob_lr_y_test)** 2))
print("R2-score: %.2f" % r2_score(y_pred , nob_lr_y_test) )

# plot slope 
plt.figure(figsize=(10,8))
plt.plot(nob_lr_X_train, nob_lr_y_train, 'o', label='original data', color='black')
plt.plot(nob_lr_X_train, nob_reg.coef_*nob_lr_X_train + nob_reg.intercept_, 'r', label='fitted line', color='red')
plt.legend()
plt.show()
 
# save the model to disk
filename = 'nob_lr_model.sav'
pickle.dump(nob_lr_model, open(filename, 'wb'))

### Decision Tree Regression

In [ ]:

nob_lrX = nob_x[["TimeSpent_minutes", "rwi", "Age_Range", "weather", "TotalSpent_RM", "Shirt_Colour", "Basket_colour"]]
nob_lrY = nob_y

nob_lrX = pd.get_dummies(nob_lrX, columns=[ "weather", "Shirt_Colour", "Basket_colour"])

nob_dtr_X_train, nob_dtr_X_test, nob_dtr_y_train, nob_dtr_y_test = train_test_split(nob_lrX, nob_lrY, test_size=0.2, random_state=1)

nob_dtr_reg = DecisionTreeRegressor(random_state=0, max_depth=100)

# fit regressor with bX and Y data
nob_dtr_reg.fit(nob_dtr_X_train, nob_dtr_y_train)

# score model
score = nob_dtr_reg.score(nob_dtr_X_test, nob_dtr_y_test)
print(score)

# make predictions
expected = nob_dtr_y_test
predicted = nob_dtr_reg.predict(nob_dtr_X_test)

print("R2-score: %.2f" % r2_score(predicted , expected) )
print("Mean absolute error: %.2f" % np.mean(np.absolute(predicted - expected)))
print("Mean sum of squares (MSE): %.2f" % np.mean((predicted - expected) ** 2))
 
# save the model to disk
filename = 'nob_dtr_model.sav'
pickle.dump(nob_dtr_reg, open(filename, 'wb'))

## Regression for Total Spent

In [ ]:
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
imp_df.head(5)

In [ ]:
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))
  
ts_fs_df = imp_df.copy()
ts_fs_df = ts_fs_df.apply(LabelEncoder().fit_transform)

ts_X = ts_fs_df.drop(['TotalSpent_RM'], axis=1)
ts_y = ts_fs_df['TotalSpent_RM']
ts_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
feat_selector = BorutaPy(ts_rf, n_estimators="auto", random_state=1)
feat_selector.fit(ts_X.values, ts_y.values.ravel())
boruta_score = ranking(list(map(float, feat_selector.ranking_)), ts_X.columns, order=-1)
boruta_score = pd.DataFrame(list(boruta_score.items()), columns=['Features', 'Score'])
boruta_score = boruta_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(boruta_score.head(10))

print('---------Bottom 10----------')
display(boruta_score.tail(10))

sns_boruta_plot = sns.catplot(x="Score", y="Features", data = boruta_score[:10], kind = "bar", height=14, aspect=2, palette='coolwarm')
plt.title("Total Spent Boruta Top 10 Features")
sns_boruta_plot.figure.savefig('TotalSpent_Boruta.png')

pd.DataFrame.to_csv(boruta_score.head(10), 'TotalSpent_Boruta_Top10.csv')

#### SVR for total spent

In [ ]:
# train test split
ts_svr_X_train, ts_svr_X_test, ts_svr_y_train, ts_svr_y_test = train_test_split(ts_X, ts_y, test_size=0.2, random_state=1)

ts_reg = SVR(kernel = 'rbf')

ts_reg.fit(ts_svr_X_train, ts_svr_y_train)
y_pred = ts_reg.predict(ts_svr_X_test)

print("Mean Absolute Error: {:.3f}".format(mean_absolute_error(ts_svr_y_test, y_pred)))
print("Mean Squared Error: {:.3f}".format(mean_squared_error(ts_svr_y_test, y_pred)))
print("R2-score: {:.3f}".format(r2_score(ts_svr_y_test, y_pred) ))
print("accuracy_score: {:.3f}".format(ts_reg.score(ts_svr_X_test, ts_svr_y_test)))

gammas = [0.1, 1, 10, 100]
svr_mae = [] 
svr_acc = []
scr_mse = []
for gamma in gammas:
    ts_reg = SVR(kernel = 'rbf', gamma = gamma)
    ts_reg.fit(ts_svr_X_train, ts_svr_y_train)
    svr_y_pred = ts_reg.predict(ts_svr_X_test)
    svr_mae.append(mean_absolute_error(ts_svr_y_test, svr_y_pred))
    svr_acc.append(ts_reg.score(ts_svr_X_test, ts_svr_y_test)) 
    scr_mse.append(mean_squared_error(ts_svr_y_test, svr_y_pred))
    
plt.plot(gammas, svr_mae)
plt.xlabel('gamma')
plt.ylabel('MAE')
plt.title('gamma vs MAE').figure.savefig('TotalSpent_SVR_Gamma_vs_Mae.png') 
plt.show()

plt.plot(gammas, svr_acc)
plt.xlabel('gamma')
plt.ylabel('Accuracy')
plt.title('gamma vs Accuracy').figure.savefig('TotalSpent_SVR_Gamma_vs_Accuracy.png')
plt.show()

plt.plot(gammas, scr_mse)
plt.xlabel('gamma')
plt.ylabel('MSE')
plt.title('gamma vs MSE').figure.savefig('TotalSpent_SVR_Gamma_vs_MSE.png')
plt.show()

# save the model to disk
filename = 'ts_svr_model.sav'
pickle.dump(ts_reg, open(filename, 'wb'))

## Regression for Customer Numbers

#### Group Datasets

In [ ]:
imp_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')
imp_df.head(5)

numcust = imp_df.groupby(['Day', 'Time', 'weather']).size().reset_index(name='numcust')
print(len(numcust))
numcust.head(5)
cn_df = numcust
display(cn_df)

#### Decision Tree Regressor for cust num

In [ ]:

rtX = cn_df.drop(['numcust'], axis=1)
# one hot encoding
rtX = pd.get_dummies(rtX, columns=["Day","Time", "weather"])
rtY = cn_df[['numcust']]

rtx_train, rtx_test, rty_train, rty_test = train_test_split(rtX, rtY, test_size=0.2, random_state=1)

print(rtx_train.shape)
print(rtx_test.shape)

# create decision tree regressor object
regressor = DecisionTreeRegressor(random_state=0)

# fit regressor with bX and Y data
regressor.fit(rtx_train, rty_train)

# score model
score = regressor.score(rtx_test, rty_test)
print(score)

# make predictions
expected = rty_test
predicted = regressor.predict(rtx_test)

print("R2-score: %.2f" % r2_score(predicted , expected) )
# calculate RMSE
rmse = sqrt(mean_squared_error(expected, predicted))
print('RMSE: %.3f' % rmse)
# calculate MAE
mae = mean_absolute_error(expected, predicted)
print('MAE: %.3f' % mae)
  

# plot rmse and mae
plt.figure(figsize=(10, 5))
plt.plot(expected, 'o', color='green')
plt.plot(predicted, 'o', color='orange')
plt.plot([0, 100], [0, 100], '--', color='red', linewidth=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

depth = 30
max_depth = [i for i in range(1,depth+1)] 
dtr_mae = [] 
for i in range(1,depth+1):
    dtr = DecisionTreeRegressor(max_depth=i)
    dtr.fit(rtx_train, rty_train)
    dtr_y_pred = dtr.predict(rtx_test)
    dtr_mae.append(mean_absolute_error(rty_test, dtr_y_pred))
    
plt.plot(max_depth, dtr_mae)
plt.xlabel('max_depth')
plt.ylabel('MAE')
plt.title('max_depth vs MAE').figure.savefig('numcust_dtr_depth_vs_mae.png') 
plt.show()

# save model to disk
filename = 'numcust_dtr_model.sav'
pickle.dump(regressor, open(filename, 'wb'))

### Making a prediction

In [ ]:
test_row = rtx_test.iloc[12]
test_row = test_row.values.reshape(1, -1)
test_row = pd.DataFrame(test_row, columns=rtx_test.columns)
# default values to 0
for col in test_row.columns:
    test_row[col].values[:] = 0
    
test_row['Day_Monday'] = 1
test_row['Time_Evening'] = 1
test_row['weather_Clear'] = 1

display(test_row)

# make a prediction
yhat = regressor.predict(test_row)
print('Predicted: %.3f' % yhat[0])

# Classification Models

### Classification for Wash Item

### Wash Item Dataset Preparation

In [ ]:
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))
  
wi_df = pd.read_csv('./dataset_w_weather&rwi&city.csv')

### Wash Item Feature Selection (Boruta & RFE)

In [ ]:
wi_df = wi_df.apply(LabelEncoder().fit_transform)

wi_x = wi_df.drop(['Wash_Item'], axis=1)
wi_y = wi_df['Wash_Item']

wi_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
wi_feat_selector = BorutaPy(wi_rf, n_estimators="auto", random_state=1)
wi_feat_selector.fit(wi_x.values, wi_y.values.ravel())

wi_boruta_score = ranking(list(map(float, wi_feat_selector.ranking_)), wi_x.columns, order=-1)
wi_boruta_score = pd.DataFrame(list(wi_boruta_score.items()), columns=['Features', 'Score'])
wi_boruta_score = wi_boruta_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(wi_boruta_score.head(10))

print('---------Bottom 10----------')
display(wi_boruta_score.tail(10))
pd.DataFrame.to_csv(wi_boruta_score.head(10), 'WashItem_Boruta_Top10.csv')

sns_wi_boruta_plot = sns.catplot(x="Score", y="Features", data = wi_boruta_score[0:10], kind = "bar", height=14, aspect=1.9, palette='coolwarm')
plt.title("Wash_Item Boruta Top-10 Features")
sns_wi_boruta_plot.figure.savefig('Wash_Item_Boruta.png')

In [ ]:
wi_rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5, n_estimators=100)
wi_rf.fit(wi_x, wi_y)
wi_rfe = RFECV(wi_rf, min_features_to_select = 1, cv = 3)
wi_rfe.fit(wi_x, wi_y)

wi_rfe_score = ranking(list(map(float, wi_rfe.ranking_)), wi_x.columns, order=-1)
wi_rfe_score = pd.DataFrame(list(wi_rfe_score.items()), columns=['Features', 'Score'])
wi_rfe_score = wi_rfe_score.sort_values("Score", ascending = False)

print('---------Top 10----------')
display(wi_rfe_score.head(10))

print('---------Bottom 10----------')
display(wi_rfe_score.tail(10))
pd.DataFrame.to_csv(wi_rfe_score.head(10), 'WashItem_RFE_Top10.csv')

sns_wi_rfe_plot = sns.catplot(x="Score", y="Features", data = wi_rfe_score[0:10], kind = "bar", height=14, aspect=1.9, palette='coolwarm')
plt.title("WashItem RFE Top-10 Features")
sns_wi_rfe_plot.figure.savefig('WashItem_RFE.png')

In [ ]:
wi_X = wi_df[["Month", "Date", "latitude" , "weather", "rwi", "Kids_Category", "Age_Range", "Pants_Colour", "TimeSpent_minutes"]]
wi_y = wi_df['Wash_Item']

X_wi_train, X_wi_test, y_wi_train, y_wi_test = train_test_split(wi_X, wi_y, test_size = 0.2, random_state = 10) 

#### Naive Bayes

In [ ]:

wi_nb = GaussianNB()
wi_nb.fit(X_wi_train, y_wi_train)
y_pred = wi_nb.predict(X_wi_test)
filename = 'Wash_Item_NB.sav'
joblib.dump(wi_nb, filename)

print("Accuracy on training set: {:.3f}".format(wi_nb.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_nb.score(X_wi_test, y_wi_test)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

confusion_matrix = confusion_matrix(y_wi_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()

prob_NB = wi_nb.predict_proba(X_wi_test)
report = classification_report(digits=6,y_true=y_wi_test, y_pred=wi_nb.predict(X_wi_test), output_dict=True)
result = pd.DataFrame(report).transpose()
print(result)

fpr = {}
tpr = {}
thresh = {}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_nb.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_wi_test, prob_NB[:,i], pos_label=i)
    c = classes[i]
    df_aux = X_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_wi_test]
    df_aux['prob'] = prob_NB[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('WASHITEM_NB_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    
# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('WASHITEM_NB_ROC.png')

# save the model to disk
filename = 'WASHITEM_NB.sav'
pickle.dump(wi_nb, open(filename, 'wb'))

#### Random Forest

In [ ]:
wi_X = wi_df[["Month", "Date", "latitude" , "weather", "rwi", "Kids_Category", "Age_Range", "Pants_Colour", "TimeSpent_minutes"]]
wi_y = wi_df['Wash_Item']

X_wi_train, X_wi_test, y_wi_train, y_wi_test = train_test_split(wi_X, wi_y, test_size = 0.2, random_state = 10) 

In [ ]:
wi_rf = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
wi_rf.fit(X_wi_train, y_wi_train)
y_pred = wi_rf.predict(X_wi_test)

print("Accuracy on training set: {:.3f}".format(wi_rf.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_rf.score(X_wi_test, y_wi_test)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

# confusion_matrix = confusion_matrix(y_wi_test, y_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

prob_RF = wi_rf.predict_proba(X_wi_test)

report = classification_report(digits=6,y_true=y_wi_test,y_pred=wi_rf.predict(X_wi_test))
print(report)

fpr = {}
tpr = {}
thresh ={}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_rf.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_wi_test, prob_RF[:,i], pos_label=i)
    c = classes[i]
    df_aux = X_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_wi_test]
    df_aux['prob'] = prob_RF[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('WASHITEM_RF_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")

# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('WASHITEM_RF_ROC.png')

# save the model to disk
filename = 'WASHITEM_RF.sav'
pickle.dump(wi_rf, open(filename, 'wb'))

#### KNN

In [ ]:
wi_X = wi_df[["Month", "Date", "latitude" , "weather", "rwi", "Kids_Category", "Age_Range", "Pants_Colour", "TimeSpent_minutes"]]
wi_y = wi_df['Wash_Item']

X_wi_train, X_wi_test, y_wi_train, y_wi_test = train_test_split(wi_X, wi_y, test_size = 0.2, random_state = 10) 

In [ ]:

wi_KNN = KNeighborsClassifier(n_neighbors=2)
wi_KNN.fit(X_wi_train, y_wi_train)
y_pred = wi_KNN.predict(X_wi_test)

print("Accuracy on training set: {:.3f}".format(wi_KNN.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_KNN.score(X_wi_test, y_wi_test)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

# confusion_matrix = confusion_matrix(y_wi_test, y_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

with open('WASHITEM_KNN_ACC.txt', 'w') as f:
    f.write("Accuracy on training set: {:.3f}\n".format(wi_KNN.score(X_wi_train, y_wi_train)))
    f.write("Accuracy on test set: {:.3f}\n".format(wi_KNN.score(X_wi_test, y_wi_test)))
    f.write('Mjority classifier Confusion Matrix\n')
    f.write(str(confusion_matrix))
    
prob_KNN = wi_KNN.predict_proba(X_wi_test)

report = classification_report(digits=6,y_true=y_wi_test, y_pred=wi_KNN.predict(X_wi_test))
print(report)

fpr = {}
tpr = {}
thresh ={}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_KNN.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_wi_test, prob_KNN[:,i], pos_label=i)
    c = classes[i]
    df_aux = X_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_wi_test]
    df_aux['prob'] = prob_KNN[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('WASHITEM_KNN_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")

# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('WASHITEM_KNN_ROC.png')
 
# save the model to disk
filename = 'WASHITEM_KNN.sav'
pickle.dump(wi_KNN, open(filename, 'wb'))

##### RF TUNING

In [ ]:

wi_rf = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
grid_values = {'max_depth': [3, 5, 10, 20, 50]}
grid_rf_acc = GridSearchCV(wi_rf, param_grid = grid_values, scoring = 'accuracy')
grid_rf_acc.fit(X_wi_train, y_wi_train)

# plot the results
plt.figure()
plt.plot(grid_rf_acc.cv_results_['param_max_depth'],
          grid_rf_acc.cv_results_['mean_test_score'], label='accuracy')
plt.xlabel('max_depth')
plt.ylabel('Cross-validation accuracy')
plt.legend()
plt.show()

# find best parameters
print('Grid best parameter (max. accuracy): ', grid_rf_acc.best_params_)
print('Grid best score (accuracy): ', grid_rf_acc.best_score_)

wi_rf.fit(X_wi_train, y_wi_train)
y_pred = wi_rf.predict(X_wi_test)

print("Accuracy on training set: {:.3f}".format(wi_rf.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_rf.score(X_wi_test, y_wi_test)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

# confusion_matrix = confusion_matrix(y_wi_test, y_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

prob_RF = wi_rf.predict_proba(X_wi_test)

report = classification_report(digits=6,y_true=y_wi_test, y_pred=wi_rf.predict(X_wi_test))
print(report)

fpr = {}
tpr = {}
thresh ={}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_rf.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_wi_test, prob_RF[:,i], pos_label=i)
    c = classes[i]
    df_aux = X_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_wi_test]
    df_aux['prob'] = prob_RF[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('WASH_ITEM_RF_1_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")

# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('WASHITEM_RF_1_ROC.png')

#### SVM

In [ ]:
wi_svc = SVC(probability=True, random_state=1, kernel='rbf', C=10, gamma=0.1)
wi_svc.fit(X_wi_train, y_wi_train)
y_pred = wi_svc.predict(X_wi_test)

print("Accuracy on training set: {:.3f}".format(wi_svc.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_svc.score(X_wi_test, y_wi_test)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

# confusion_matrix = confusion_matrix(y_wi_test, y_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

prob_svc = wi_svc.predict_proba(X_wi_test)
report = classification_report(digits=6,y_true=y_wi_test, y_pred=wi_svc.predict(X_wi_test))
print(report)

fpr = {}
tpr = {}
thresh ={}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_svc.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_wi_test, prob_svc[:,i], pos_label=i)
    c = classes[i]
    df_aux = X_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_wi_test]
    df_aux['prob'] = prob_svc[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('WASH_ITEM_SVC_1_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")

# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('WASHITEM_SVC_1_ROC.png')

# save the model to disk 
filename = 'WASH_ITEM_SVC_1.sav'
pickle.dump(wi_svc, open(filename, 'wb'))

#### Ensemble

In [ ]:
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = KNeighborsClassifier(n_neighbors=3)
estimators.append(('knn', model2))
model3 = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=10)

estimators.append(('rf', model3))
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X_wi_train, y_wi_train)
y_pred = ensemble.predict(X_wi_test)

print("Accuracy on training set: {:.3f}".format(ensemble.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(ensemble.score(X_wi_test, y_wi_test)))
print('Accuracy Score : ' + str(accuracy_score(y_wi_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

prob_ESB = ensemble.predict_proba(X_wi_test)

sk_report = classification_report(digits=6,y_true=y_wi_test, y_pred=ensemble.predict(X_wi_test))
print(sk_report)

# save the model to disk
filename = 'WASH_ITEM_ESB_VOT.sav'
pickle.dump(ensemble, open(filename, 'wb'))

In [ ]:
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
level1 = LogisticRegression()
stacked_model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

models = dict()
models['lr'] = LogisticRegression()
models['knn'] = KNeighborsClassifier()
models['cart'] = DecisionTreeClassifier()
models['svm'] = SVC()
models['bayes'] = GaussianNB()
models['stacking'] = stacked_model

def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X_wi_test, y_wi_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

results, names = list(), list() 
for name, model in models.items():
  scores = evaluate_model(model, X_wi_train, y_wi_train)
  results.append(scores)
  names.append(name)
  print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

plt.boxplot(results, labels=names, showmeans=True)
plt.show()

# save the model to disk
filename = 'WASH_ITEM_ESB_STACKING.sav'
pickle.dump(stacked_model, open(filename, 'wb'))

#### SMOTE

In [ ]:
sm = SMOTE(random_state=0)
XSMOTE_wi_train, XSMOTE_wi_test, ySMOTE_wi_train, ySMOTE_wi_test = train_test_split(wi_X, wi_y.values.ravel(), test_size=0.4, random_state=10)
columns = XSMOTE_wi_train.columns
sm_data_X, sm_data_y = sm.fit_resample(XSMOTE_wi_train, ySMOTE_wi_train)
os_data_X = pd.DataFrame(data=sm_data_X, columns=columns)
os_data_y = pd.DataFrame(data=sm_data_y, columns=['Wash_Item'])


print("Oversampled  data has {} rows and {} columns".format(len(os_data_X),len(os_data_X.columns)))
print("Proportion of 1 data in oversampled data is ",len(os_data_y[os_data_y['Wash_Item']==0])/len(os_data_X))
print("Proportion of 2 data in oversampled data is ",len(os_data_y[os_data_y['Wash_Item']==1])/len(os_data_X))

In [ ]:
XSMOTE_wi_train, XSMOTE_wi_test, ySMOTE_wi_train, ySMOTE_wi_test = train_test_split(os_data_X, os_data_y, test_size=0.4, random_state=10)
ySMOTE_wi_test = ySMOTE_wi_test.squeeze()

In [ ]:
wi_Smote_RF = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=100)
wi_Smote_RF.fit(XSMOTE_wi_train, ySMOTE_wi_train)
ySMOTE_pred = wi_Smote_RF.predict(XSMOTE_wi_test)

# Calculate the overall accuracy on test set 
print("Accuracy on training set: {:.3f}".format(ensemble.score(X_wi_train, y_wi_train)))
print("Accuracy on test set: {:.3f}".format(ensemble.score(X_wi_test, y_wi_test)))
print('Accuracy Score : ' + str(accuracy_score(y_wi_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_wi_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_wi_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_wi_test,y_pred)))

# confusion_matrix = confusion_matrix(ySMOTE_wi_test, ySMOTE_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

prob_RF = wi_Smote_RF.predict_proba(XSMOTE_wi_test)

report = classification_report(digits=6,y_true=ySMOTE_wi_test, y_pred=wi_Smote_RF.predict(XSMOTE_wi_test))
print(report)

fpr = {}
tpr = {}
thresh ={}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_Smote_RF.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(ySMOTE_wi_test, prob_RF[:,i], pos_label=i)
    c = classes[i]
    df_aux = XSMOTE_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in ySMOTE_wi_test]
    df_aux['prob'] = prob_RF[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('SMOTE_WASH_ITEM_RF_1_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")

# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='-')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('SMOTE_WASHITEM_RF_1_ROC.png')

# save the model to disk
filename = 'SMOTE_WASH_ITEM_RF_1.sav'

In [ ]:
wi_nb_Smote = GaussianNB()
wi_nb_Smote.fit(XSMOTE_wi_train, ySMOTE_wi_train)
ySMOTE_pred = wi_nb_Smote.predict(XSMOTE_wi_test)
filename = 'SMOTE_Wash_Item_NB.sav'
joblib.dump(wi_nb_Smote, filename)

print("Accuracy on training set: {:.3f}".format(wi_nb_Smote.score(XSMOTE_wi_train, ySMOTE_wi_train)))
print("Accuracy on test set: {:.3f}".format(wi_nb_Smote.score(XSMOTE_wi_test, ySMOTE_wi_test)))
print('Precision Score : ' + str(precision_score(ySMOTE_wi_test,ySMOTE_pred)))
print('Recall Score : ' + str(recall_score(ySMOTE_wi_test,ySMOTE_pred)))
print('F1 Score : ' + str(f1_score(ySMOTE_wi_test,ySMOTE_pred)))

# confusion_matrix = confusion_matrix(ySMOTE_wi_test, ySMOTE_pred)
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()

prob_NB = wi_nb_Smote.predict_proba(XSMOTE_wi_test)
report = classification_report(digits=6,y_true=ySMOTE_wi_test, y_pred=wi_nb_Smote.predict(XSMOTE_wi_test), output_dict=True)
result = pd.DataFrame(report).transpose()
print(result)

fpr = {}
tpr = {}
thresh = {}
n_class = wi_df['Wash_Item'].nunique()
classes = wi_nb_Smote.classes_
roc_auc_ovr = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(ySMOTE_wi_test, prob_NB[:,i], pos_label=i)
    c = classes[i]
    df_aux = XSMOTE_wi_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in ySMOTE_wi_test]
    df_aux['prob'] = prob_NB[:, i]
    df_aux = df_aux.reset_index(drop = True)
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])

avg_roc_auc = 0
i = 0
with open('SMOTE_WASHITEM_NB_AUC.txt', 'w') as f:
    for k in roc_auc_ovr:
        avg_roc_auc += roc_auc_ovr[k]
        i += 1
        f.write(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}\n")
        print(f"Class {k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
    print(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    f.write(f"Average ROC AUC OvR: {avg_roc_auc/i:.4f}")
    
# plotting    
plt.plot(fpr[0], tpr[0], color='red', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], color='orange', label='Class 1 vs Rest')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best').figure.savefig('SMOTE_WASHITEM_NB_ROC.png')

# save the model to disk
filename = 'SMOTE_WASH_ITEM_NB.sav'
pickle.dump(wi_nb_Smote, open(filename, 'wb'))

# Clustering Models

In [ ]:
imp_df.dtypes

In [ ]:
coordinate = plt.scatter(imp_df['latitude'],imp_df['longitude'])
plt.xlabel('city')
plt.ylabel('rwi')
coordinate.figure.savefig('lat_lng_scatter.png')
coordinate

In [ ]:
X = imp_df[['longitude', 'latitude']]
sil = []
kmax = 10

for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(X)
  labels = kmeans.labels_
  sil.append(silhouette_score(X, labels, metric = 'euclidean'))
  
plt.plot(range(2, kmax+1), sil, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('Silhouette Method For Optimal k')
plt.show()
plt.savefig('long_lat_silhouette.png')

In [ ]:
model = KMeans(n_clusters = 3, verbose=0)
model.fit(X)
clusters = model.fit_predict(X)

df_clusters = imp_df.copy()
df_clusters['Clusters'] = clusters 
plt.scatter(df_clusters['longitude'],df_clusters['latitude'],c=df_clusters['Clusters'],cmap='autumn')
plt.savefig('long_lat_clustered.png')

# save the model to disk
filename = 'long_lat_cluster.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# min max scaler
scaler = MinMaxScaler()
scaler.fit(imp_df[['Age_Range']])
imp_df['Age_Range'] = scaler.transform(imp_df[['Age_Range']])
imp_df.head(5)

coordinate = plt.scatter(imp_df['rwi'],imp_df['Age_Range'])
plt.xlabel('city')
plt.ylabel('rwi')
coordinate.figure.savefig('city_rwi_scatter.png')
coordinate

In [ ]:
sil = []
kmax = 10
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(X)
  labels = kmeans.labels_
  sil.append(silhouette_score(X, labels, metric = 'euclidean'))
  
plt.plot(range(2, kmax+1), sil, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('Silhouette Method For Optimal k')
plt.show()
plt.savefig('age_range_rwi_silhouette.png')


In [ ]:
X = imp_df[['Age_Range', 'rwi']]

model = KMeans(n_clusters = 3, verbose=0) 
model.fit(X)
clusters = model.fit_predict(X)

df_clusters = imp_df.copy()
df_clusters['Clusters'] = clusters 
plt.scatter(df_clusters['Age_Range'],df_clusters['rwi'],c=df_clusters['Clusters'],cmap='autumn')
plt.savefig('age_range_rwi_clustered.png')

# save the model to disk
filename = 'age_range_rwi_cluster.sav' 
pickle.dump(model, open(filename, 'wb'))